In [1]:
pip install pulp pandas numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
"""
FDC Production Scheduling: Complete Sensitivity Analysis with Export
Generates tables (CSV/Excel) and graphs (PNG) for all Q2-Q9 analyses
"""

import pulp as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# Set style for all plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# =============================================================================
# OUTPUT DIRECTORY SETUP
# =============================================================================

def setup_output_dirs():
    """Create output directory structure"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_dir = f"FDC_Sensitivity_Analysis_{timestamp}"
    
    dirs = {
        'base': base_dir,
        'tables': os.path.join(base_dir, 'Tables'),
        'graphs': os.path.join(base_dir, 'Graphs'),
        'q2': os.path.join(base_dir, 'Graphs', 'Q2'),
        'q3': os.path.join(base_dir, 'Graphs', 'Q3'),
        'q4': os.path.join(base_dir, 'Graphs', 'Q4'),
        'q5': os.path.join(base_dir, 'Graphs', 'Q5'),
        'q6': os.path.join(base_dir, 'Graphs', 'Q6'),
        'q7': os.path.join(base_dir, 'Graphs', 'Q7'),
        'q8': os.path.join(base_dir, 'Graphs', 'Q8'),
        'q9': os.path.join(base_dir, 'Graphs', 'Q9'),
    }
    
    for d in dirs.values():
        os.makedirs(d, exist_ok=True)
    
    print(f"Output directory created: {base_dir}")
    return dirs

# =============================================================================
# DATA DEFINITION
# =============================================================================

demand = {
    1: [3500, 3000, 4000, 4000, 2800], 
    2: [3000, 2800, 4000, 4300, 2800],
    3: [3000, 2000, 4000, 3500, 3000], 
    4: [3000, 3000, 4000, 3800, 2800],
    5: [3000, 3000, 4000, 4000, 2800],
    6: [3500, 2500, 4000, 3800, 2500],
    7: [3500, 2500, 3800, 4000, 2500], 
    8: [3300, 3400, 3700, 4200, 2500],
    9: [3300, 3400, 0, 4500, 3000], 
    10: [3200, 3000, 0, 4500, 3000],
    11: [4500, 4000, 5000, 5000, 3800],
    12: [3000, 2800, 4000, 4300, 2800]
}

production_rates = {
    'Part1': [40, 35, None, None, None], 
    'Part2': [None, 25, 30, 35, None],
    'Part3': [None, None, None, 50, None], 
    'Part4': [60, None, None, None, 60],
    'Part5': [None, None, 45, None, 50]
}
rates_df = pd.DataFrame(production_rates, 
    index=['Machine1', 'Machine2', 'Machine3', 'Machine4', 'Machine5'])

yields_base = {'Part1': 0.60, 'Part2': 0.55, 'Part3': 0.75, 'Part4': 0.65, 'Part5': 0.60}

setup_times_data = {
    'Part1': [8, 10, None, None, None], 
    'Part2': [None, 8, 10, 8, None],
    'Part3': [None, None, None, 12, None], 
    'Part4': [8, None, None, None, 8],
    'Part5': [None, None, 24, None, 20]
}
setup_times_df = pd.DataFrame(setup_times_data, 
    index=['Machine1', 'Machine2', 'Machine3', 'Machine4', 'Machine5'])

initial_setups_base = {
    ('Machine1', 'Part1'): 1,
    ('Machine2', 'Part2'): 1, 
    ('Machine3', 'Part5'): 1,
    ('Machine4', 'Part3'): 1, 
    ('Machine5', 'Part4'): 1
}

INITIAL_CONFIGS = {
    'Config_A': {('Machine1', 'Part1'): 1, 
                 ('Machine2', 'Part2'): 1, 
                 ('Machine3', 'Part5'): 1,
                 ('Machine4', 'Part3'): 1, 
                 ('Machine5', 'Part4'): 1},
    'Config_B': {('Machine1', 'Part4'): 1, 
                 ('Machine2', 'Part1'): 1, 
                 ('Machine3', 'Part5'): 1,
                 ('Machine4', 'Part3'): 1, 
                 ('Machine5', 'Part4'): 1},
    'Config_C': {('Machine1', 'Part4'): 1, 
                 ('Machine2', 'Part1'): 1, 
                 ('Machine3', 'Part2'): 1,
                 ('Machine4', 'Part3'): 1, 
                 ('Machine5', 'Part5'): 1},
    'Config_D': {('Machine1', 'Part1'): 1, 
                 ('Machine2', 'Part1'): 1, 
                 ('Machine3', 'Part2'): 1,
                 ('Machine4', 'Part2'): 1, 
                 ('Machine5', 'Part5'): 1},
    'Config_E': {('Machine1', 'Part4'): 1, 
                 ('Machine2', 'Part2'): 1, 
                 ('Machine3', 'Part5'): 1,
                 ('Machine4', 'Part2'): 1, 
                 ('Machine5', 'Part4'): 1}
}

BASE_PARAMS = {
    'weekly_time_limit': 120, 'max_overtime': 48, 'machine_ot_cost': 30,
    'support_cost': 40, 'penalty_cost': 3, 'holding_cost': 2, 'BIG_M': 170
}

# Helper functions
def get_machines(): return ['Machine1', 'Machine2', 'Machine3', 'Machine4', 'Machine5']
def get_parts(): return ['Part1', 'Part2', 'Part3', 'Part4', 'Part5']
def get_feasible(rates):
    return [(m, p) for m in rates.index for p in rates.columns 
            if rates.loc[m, p] is not None and not pd.isna(rates.loc[m, p])]
def get_demand_dict(demand_list):
    return {f'Part{i+1}': demand_list[i] for i in range(5)}
def scale_demand(base_list, factor):
    return [int(d * factor) for d in base_list]
def scale_yields(base_yields, improvement):
    return {p: min(base_yields[p] + improvement, 1.0) for p in base_yields}
def scale_setup(base_df, reduction):
    new_data = {}
    for p in base_df.columns:
        new_data[p] = [v * (1 - reduction) if v is not None and not pd.isna(v) else None 
                      for v in base_df[p]]
    return pd.DataFrame(new_data, index=base_df.index)

# =============================================================================
# MODEL SOLVERS (Compact versions)
# =============================================================================

def solve_q2(demand_list, yields=None, setup_df=None, params=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    demand_dict = get_demand_dict(demand_list)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q2", pl.LpMinimize)
    x = {(m,p): pl.LpVariable(f"x_{m}_{p}", lowBound=0) for (m,p) in F}
    y = {(m,p): pl.LpVariable(f"y_{m}_{p}", cat='Binary') for (m,p) in F}
    OT = {m: pl.LpVariable(f"OT_{m}", lowBound=0, upBound=OT_MAX) for m in machines}
    
    model += pl.lpSum(OT[m] for m in machines)
    for p in parts:
        model += pl.lpSum(x[m,p] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) >= demand_dict[p]
    for m in machines:
        model += pl.lpSum(x[m,p] + setup_df.loc[m,p] * y[m,p] for p in parts if (m,p) in F) <= RT + OT[m]
    for (m,p) in F:
        model += x[m,p] <= M * y[m,p]
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': sum(OT[m].varValue or 0 for m in machines),
            'overtime_by_machine': {m: OT[m].varValue or 0 for m in machines}}

def solve_q3(demand_list, yields=None, setup_df=None, params=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    demand_dict = get_demand_dict(demand_list)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q3", pl.LpMinimize)
    x = {(m,p): pl.LpVariable(f"x_{m}_{p}", lowBound=0) for (m,p) in F}
    y = {(m,p): pl.LpVariable(f"y_{m}_{p}", cat='Binary') for (m,p) in F}
    OT = {m: pl.LpVariable(f"OT_{m}", lowBound=0, upBound=OT_MAX) for m in machines}
    Z = pl.LpVariable("Z", lowBound=0)
    
    model += Z
    for p in parts:
        model += pl.lpSum(x[m,p] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) >= demand_dict[p]
    for m in machines:
        model += pl.lpSum(x[m,p] + setup_df.loc[m,p] * y[m,p] for p in parts if (m,p) in F) <= RT + OT[m]
        model += OT[m] <= Z
    for (m,p) in F:
        model += x[m,p] <= M * y[m,p]
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'max_overtime': Z.varValue or 0, 'total_overtime': sum(OT[m].varValue or 0 for m in machines),
            'overtime_by_machine': {m: OT[m].varValue or 0 for m in machines}}

def solve_q4(demand_list, yields=None, setup_df=None, params=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    c_prod, c_supp = params['machine_ot_cost'], params['support_cost']
    demand_dict = get_demand_dict(demand_list)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q4", pl.LpMinimize)
    x = {(m,p): pl.LpVariable(f"x_{m}_{p}", lowBound=0) for (m,p) in F}
    y = {(m,p): pl.LpVariable(f"y_{m}_{p}", cat='Binary') for (m,p) in F}
    OT = {m: pl.LpVariable(f"OT_{m}", lowBound=0, upBound=OT_MAX) for m in machines}
    Z = pl.LpVariable("Z", lowBound=0)
    
    model += c_prod * pl.lpSum(OT[m] for m in machines) + c_supp * Z
    for p in parts:
        model += pl.lpSum(x[m,p] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) >= demand_dict[p]
    for m in machines:
        model += pl.lpSum(x[m,p] + setup_df.loc[m,p] * y[m,p] for p in parts if (m,p) in F) <= RT + OT[m]
        model += OT[m] <= Z
    for (m,p) in F:
        model += x[m,p] <= M * y[m,p]
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    total_ot = sum(OT[m].varValue or 0 for m in machines)
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': total_ot, 'max_overtime': Z.varValue or 0,
            'overtime_cost': c_prod * total_ot, 'support_cost': c_supp * (Z.varValue or 0)}

def solve_q5(demand_list, yields=None, setup_df=None, params=None, initial_setup=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    initial_setup = initial_setup or initial_setups_base
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    demand_dict = get_demand_dict(demand_list)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q5", pl.LpMinimize)
    x = {(m,p): pl.LpVariable(f"x_{m}_{p}", lowBound=0) for (m,p) in F}
    y = {(m,p): pl.LpVariable(f"y_{m}_{p}", cat='Binary') for (m,p) in F}
    OT = {m: pl.LpVariable(f"OT_{m}", lowBound=0, upBound=OT_MAX) for m in machines}
    
    model += pl.lpSum(OT[m] for m in machines)
    for p in parts:
        model += pl.lpSum(x[m,p] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) >= demand_dict[p]
    for m in machines:
        terms = [x[m,p] if initial_setup.get((m,p), 0) == 1 else x[m,p] + setup_df.loc[m,p] * y[m,p] 
                 for p in parts if (m,p) in F]
        model += pl.lpSum(terms) <= RT + OT[m]
    for (m,p) in F:
        model += x[m,p] <= M * y[m,p]
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': sum(OT[m].varValue or 0 for m in machines)}

def solve_q6(demand_list, yields=None, setup_df=None, params=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    c_prod, c_supp, penalty = params['machine_ot_cost'], params['support_cost'], params['penalty_cost']
    demand_dict = get_demand_dict(demand_list)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q6", pl.LpMinimize)
    x = {(m,p): pl.LpVariable(f"x_{m}_{p}", lowBound=0) for (m,p) in F}
    y = {(m,p): pl.LpVariable(f"y_{m}_{p}", cat='Binary') for (m,p) in F}
    OT = {m: pl.LpVariable(f"OT_{m}", lowBound=0, upBound=OT_MAX) for m in machines}
    Z = pl.LpVariable("Z", lowBound=0)
    U = {p: pl.LpVariable(f"U_{p}", lowBound=0) for p in parts}
    
    model += c_prod * pl.lpSum(OT[m] for m in machines) + c_supp * Z + penalty * pl.lpSum(U[p] for p in parts)
    for p in parts:
        model += pl.lpSum(x[m,p] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) + U[p] >= demand_dict[p]
    for m in machines:
        model += pl.lpSum(x[m,p] + setup_df.loc[m,p] * y[m,p] for p in parts if (m,p) in F) <= RT + OT[m]
        model += OT[m] <= Z
    for (m,p) in F:
        model += x[m,p] <= M * y[m,p]
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    total_ot = sum(OT[m].varValue or 0 for m in machines)
    total_short = sum(U[p].varValue or 0 for p in parts)
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': total_ot, 'max_overtime': Z.varValue or 0, 'total_shortfall': total_short,
            'overtime_cost': c_prod * total_ot, 'support_cost': c_supp * (Z.varValue or 0),
            'shortfall_cost': penalty * total_short, 'shortfall_by_part': {p: U[p].varValue or 0 for p in parts}}

def solve_q7(demand_w1, demand_w2, yields=None, setup_df=None, params=None, initial_setup=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    initial_setup = initial_setup or initial_setups_base
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    d1, d2 = get_demand_dict(demand_w1), get_demand_dict(demand_w2)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q7", pl.LpMinimize)
    x = {(m,p,t): pl.LpVariable(f"x_{m}_{p}_{t}", lowBound=0) for (m,p) in F for t in [1,2]}
    y = {(m,p,t): pl.LpVariable(f"y_{m}_{p}_{t}", cat='Binary') for (m,p) in F for t in [1,2]}
    OT = {(m,t): pl.LpVariable(f"OT_{m}_{t}", lowBound=0, upBound=OT_MAX) for m in machines for t in [1,2]}
    C = {(m,p): pl.LpVariable(f"C_{m}_{p}", cat='Binary') for (m,p) in F}
    
    model += pl.lpSum(OT[m,t] for m in machines for t in [1,2])
    for p in parts:
        model += pl.lpSum(x[m,p,1] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) >= d1[p]
        model += pl.lpSum(x[m,p,2] * rates_df.loc[m,p] * yields[p] for m in machines if (m,p) in F) >= d2[p]
    for m in machines:
        t1 = [x[m,p,1] if initial_setup.get((m,p),0)==1 else x[m,p,1]+setup_df.loc[m,p]*y[m,p,1] for p in parts if (m,p) in F]
        model += pl.lpSum(t1) <= RT + OT[m,1]
        t2 = [x[m,p,2] + setup_df.loc[m,p]*(y[m,p,2]-C[m,p]) for p in parts if (m,p) in F]
        model += pl.lpSum(t2) <= RT + OT[m,2]
    for (m,p) in F:
        model += x[m,p,1] <= M*y[m,p,1]; model += x[m,p,2] <= M*y[m,p,2]
        model += C[m,p] <= y[m,p,1]; model += C[m,p] <= y[m,p,2]
    for m in machines:
        model += pl.lpSum(C[m,p] for p in parts if (m,p) in F) <= 1
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': sum(OT[m,t].varValue or 0 for m in machines for t in [1,2]),
            'overtime_week1': sum(OT[m,1].varValue or 0 for m in machines),
            'overtime_week2': sum(OT[m,2].varValue or 0 for m in machines),
            'carryovers': sum(1 for (m,p) in F if (C[m,p].varValue or 0) > 0.5)}

def solve_q8(demand_w1, demand_w2, yields=None, setup_df=None, params=None, initial_setup=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    initial_setup = initial_setup or initial_setups_base
    RT, OT_MAX, M, h = params['weekly_time_limit'], params['max_overtime'], params['BIG_M'], params['holding_cost']
    d1, d2 = get_demand_dict(demand_w1), get_demand_dict(demand_w2)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q8", pl.LpMinimize)
    x = {(m,p,t): pl.LpVariable(f"x_{m}_{p}_{t}", lowBound=0) for (m,p) in F for t in [1,2]}
    y = {(m,p,t): pl.LpVariable(f"y_{m}_{p}_{t}", cat='Binary') for (m,p) in F for t in [1,2]}
    OT = {(m,t): pl.LpVariable(f"OT_{m}_{t}", lowBound=0, upBound=OT_MAX) for m in machines for t in [1,2]}
    C = {(m,p): pl.LpVariable(f"C_{m}_{p}", cat='Binary') for (m,p) in F}
    I = {p: pl.LpVariable(f"I_{p}", lowBound=0) for p in parts}
    
    model += pl.lpSum(OT[m,t] for m in machines for t in [1,2]) + h * pl.lpSum(I[p] for p in parts)
    for p in parts:
        model += pl.lpSum(x[m,p,1]*rates_df.loc[m,p]*yields[p] for m in machines if (m,p) in F) >= d1[p] + I[p]
        model += pl.lpSum(x[m,p,2]*rates_df.loc[m,p]*yields[p] for m in machines if (m,p) in F) + I[p] >= d2[p]
    for m in machines:
        t1 = [x[m,p,1] if initial_setup.get((m,p),0)==1 else x[m,p,1]+setup_df.loc[m,p]*y[m,p,1] for p in parts if (m,p) in F]
        model += pl.lpSum(t1) <= RT + OT[m,1]
        t2 = [x[m,p,2] + setup_df.loc[m,p]*(y[m,p,2]-C[m,p]) for p in parts if (m,p) in F]
        model += pl.lpSum(t2) <= RT + OT[m,2]
    for (m,p) in F:
        model += x[m,p,1] <= M*y[m,p,1]; model += x[m,p,2] <= M*y[m,p,2]
        model += C[m,p] <= y[m,p,1]; model += C[m,p] <= y[m,p,2]
    for m in machines:
        model += pl.lpSum(C[m,p] for p in parts if (m,p) in F) <= 1
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    total_inv = sum(I[p].varValue or 0 for p in parts)
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': sum(OT[m,t].varValue or 0 for m in machines for t in [1,2]),
            'overtime_week1': sum(OT[m,1].varValue or 0 for m in machines),
            'overtime_week2': sum(OT[m,2].varValue or 0 for m in machines),
            'total_inventory': total_inv, 'inventory_cost': h * total_inv,
            'inventory_by_part': {p: I[p].varValue or 0 for p in parts}}

def solve_q9(demand_w1, demand_w2, yields=None, setup_df=None, params=None, initial_setup=None):
    yields = yields or yields_base
    setup_df = setup_df if setup_df is not None else setup_times_df
    params = params or BASE_PARAMS
    initial_setup = initial_setup or initial_setups_base
    RT, OT_MAX, M = params['weekly_time_limit'], params['max_overtime'], params['BIG_M']
    c_prod, c_supp, h, penalty = params['machine_ot_cost'], params['support_cost'], params['holding_cost'], params['penalty_cost']
    d1, d2 = get_demand_dict(demand_w1), get_demand_dict(demand_w2)
    F = get_feasible(rates_df)
    machines, parts = get_machines(), get_parts()
    
    model = pl.LpProblem("Q9", pl.LpMinimize)
    x = {(m,p,t): pl.LpVariable(f"x_{m}_{p}_{t}", lowBound=0) for (m,p) in F for t in [1,2]}
    y = {(m,p,t): pl.LpVariable(f"y_{m}_{p}_{t}", cat='Binary') for (m,p) in F for t in [1,2]}
    OT = {(m,t): pl.LpVariable(f"OT_{m}_{t}", lowBound=0, upBound=OT_MAX) for m in machines for t in [1,2]}
    Z = pl.LpVariable("Z", lowBound=0)
    C = {(m,p): pl.LpVariable(f"C_{m}_{p}", cat='Binary') for (m,p) in F}
    I = {p: pl.LpVariable(f"I_{p}", lowBound=0) for p in parts}
    U = {(p,t): pl.LpVariable(f"U_{p}_{t}", lowBound=0) for p in parts for t in [1,2]}
    
    model += c_prod*pl.lpSum(OT[m,t] for m in machines for t in [1,2]) + c_supp*Z + h*pl.lpSum(I[p] for p in parts) + penalty*pl.lpSum(U[p,t] for p in parts for t in [1,2])
    for p in parts:
        model += pl.lpSum(x[m,p,1]*rates_df.loc[m,p]*yields[p] for m in machines if (m,p) in F) + U[p,1] >= d1[p] + I[p]
        model += pl.lpSum(x[m,p,2]*rates_df.loc[m,p]*yields[p] for m in machines if (m,p) in F) + I[p] + U[p,2] >= d2[p]
    for m in machines:
        for t in [1,2]: model += OT[m,t] <= Z
        t1 = [x[m,p,1] if initial_setup.get((m,p),0)==1 else x[m,p,1]+setup_df.loc[m,p]*y[m,p,1] for p in parts if (m,p) in F]
        model += pl.lpSum(t1) <= RT + OT[m,1]
        t2 = [x[m,p,2] + setup_df.loc[m,p]*(y[m,p,2]-C[m,p]) for p in parts if (m,p) in F]
        model += pl.lpSum(t2) <= RT + OT[m,2]
    for (m,p) in F:
        model += x[m,p,1] <= M*y[m,p,1]; model += x[m,p,2] <= M*y[m,p,2]
        model += C[m,p] <= y[m,p,1]; model += C[m,p] <= y[m,p,2]
    for m in machines:
        model += pl.lpSum(C[m,p] for p in parts if (m,p) in F) <= 1
    
    model.solve(pl.PULP_CBC_CMD(msg=0))
    total_ot = sum(OT[m,t].varValue or 0 for m in machines for t in [1,2])
    total_inv = sum(I[p].varValue or 0 for p in parts)
    total_short = sum(U[p,t].varValue or 0 for p in parts for t in [1,2])
    return {'status': pl.LpStatus[model.status], 'objective': pl.value(model.objective),
            'total_overtime': total_ot, 'max_overtime': Z.varValue or 0,
            'total_inventory': total_inv, 'total_shortfall': total_short,
            'overtime_cost': c_prod*total_ot, 'support_cost': c_supp*(Z.varValue or 0),
            'inventory_cost': h*total_inv, 'shortfall_cost': penalty*total_short,
            'shortfall_by_part': {p: (U[p,1].varValue or 0)+(U[p,2].varValue or 0) for p in parts}}

# =============================================================================
# Q2 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q2_all(dirs, week=1):
    """Run all Q2 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ2 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Demand Overall
    factors = [0.80, 0.90, 1.00, 1.10, 1.20]
    results = []
    for f in factors:
        r = solve_q2(scale_demand(demand[week], f))
        results.append({'Demand_Factor': f, 'Total_OT': r['total_overtime'], 'Objective': r['objective']})
    df = pd.DataFrame(results)
    all_tables['Q2_Demand_Overall'] = df
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(df['Demand_Factor']*100, df['Total_OT'], 'bo-', linewidth=2, markersize=8)
    ax.set_xlabel('Demand Factor (%)', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q2: Total Overtime vs Demand Factor', fontsize=14)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q2'], 'Q2_Demand_Overall.png'), dpi=150)
    plt.close()
    
    # 2. Demand by Part
    results = []
    base = solve_q2(demand[week])
    for i, part in enumerate(get_parts()):
        for factor in [0.80, 1.00, 1.20]:
            d = demand[week].copy()
            d[i] = int(d[i] * factor)
            r = solve_q2(d)
            results.append({'Part': part, 'Factor': factor, 'Total_OT': r['total_overtime'],
                           'Change': r['total_overtime'] - base['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q2_Demand_ByPart'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    pivot = df.pivot(index='Part', columns='Factor', values='Total_OT')
    pivot.plot(kind='bar', ax=ax)
    ax.set_xlabel('Part', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q2: Overtime by Part and Demand Factor', fontsize=14)
    ax.legend(title='Demand Factor')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q2'], 'Q2_Demand_ByPart.png'), dpi=150)
    plt.close()
    
    # 3. Yield
    improvements = [0.00, 0.05, 0.10, 0.15, 0.20]
    results = []
    base = solve_q2(demand[week])
    for imp in improvements:
        r = solve_q2(demand[week], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Total_OT': r['total_overtime'],
                       'Savings': base['total_overtime'] - r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q2_Yield'] = df
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(df['Yield_Improvement']*100, df['Total_OT'], color='green', alpha=0.7)
    ax.set_xlabel('Yield Improvement (%)', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q2: Overtime vs Yield Improvement', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q2'], 'Q2_Yield.png'), dpi=150)
    plt.close()
    
    # 4. Setup Times
    reductions = [0.00, 0.25, 0.50]
    results = []
    for red in reductions:
        r = solve_q2(demand[week], setup_df=scale_setup(setup_times_df, red))
        results.append({'Setup_Reduction': red, 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q2_Setup'] = df
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar([f"{int(r*100)}%" for r in reductions], df['Total_OT'], color='orange', alpha=0.7)
    ax.set_xlabel('Setup Time Reduction', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q2: Overtime vs Setup Time Reduction', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q2'], 'Q2_Setup.png'), dpi=150)
    plt.close()
    
    # 5. Max Overtime
    ot_values = [36, 42, 48, 54, 60]
    results = []
    for ot in ot_values:
        p = BASE_PARAMS.copy()
        p['max_overtime'] = ot
        p['BIG_M'] = 120 + ot + 2
        r = solve_q2(demand[week], params=p)
        results.append({'Max_OT_Allowed': ot, 'Total_OT_Used': r['total_overtime'], 'Status': r['status']})
    df = pd.DataFrame(results)
    all_tables['Q2_MaxOT'] = df
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(df['Max_OT_Allowed'], df['Total_OT_Used'], 'rs-', linewidth=2, markersize=8)
    ax.plot([36, 60], [36, 60], 'k--', alpha=0.3, label='y=x (binding)')
    ax.set_xlabel('Max Overtime Allowed (hrs)', fontsize=12)
    ax.set_ylabel('Total Overtime Used (hrs)', fontsize=12)
    ax.set_title('Q2: Overtime Used vs Max Allowed', fontsize=14)
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q2'], 'Q2_MaxOT.png'), dpi=150)
    plt.close()
    
    return all_tables

# =============================================================================
# Q3 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q3_all(dirs, week=1):
    """Run all Q3 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ3 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Demand
    factors = [0.80, 0.90, 1.00, 1.10, 1.20]
    results = []
    for f in factors:
        r = solve_q3(scale_demand(demand[week], f))
        results.append({'Demand_Factor': f, 'Max_OT': r['max_overtime'], 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q3_Demand'] = df
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(df['Demand_Factor']*100, df['Max_OT'], 'bo-', linewidth=2, markersize=8, label='Max OT')
    ax.plot(df['Demand_Factor']*100, df['Total_OT'], 'g^--', linewidth=2, markersize=8, label='Total OT')
    ax.set_xlabel('Demand Factor (%)', fontsize=12)
    ax.set_ylabel('Overtime (hrs)', fontsize=12)
    ax.set_title('Q3: Overtime vs Demand Factor', fontsize=14)
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q3'], 'Q3_Demand.png'), dpi=150)
    plt.close()
    
    # 2. Setup
    reductions = [0.00, 0.25, 0.50]
    results = []
    for red in reductions:
        r = solve_q3(demand[week], setup_df=scale_setup(setup_times_df, red))
        results.append({'Setup_Reduction': red, 'Max_OT': r['max_overtime'], 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q3_Setup'] = df
    
    # 3. Yield
    improvements = [0.00, 0.05, 0.10]
    results = []
    for imp in improvements:
        r = solve_q3(demand[week], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Max_OT': r['max_overtime'], 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q3_Yield'] = df
    
    # 4. Compare to Q2
    factors = [0.90, 1.00, 1.10]
    results = []
    for f in factors:
        d = scale_demand(demand[week], f)
        r2 = solve_q2(d)
        r3 = solve_q3(d)
        results.append({'Demand_Factor': f, 'Q2_Total_OT': r2['total_overtime'], 'Q3_Total_OT': r3['total_overtime'],
                       'Q3_Max_OT': r3['max_overtime'], 'Balancing_Cost': r3['total_overtime'] - r2['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q3_vs_Q2'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(factors))
    width = 0.35
    ax.bar(x - width/2, df['Q2_Total_OT'], width, label='Q2 Total OT', color='blue', alpha=0.7)
    ax.bar(x + width/2, df['Q3_Total_OT'], width, label='Q3 Total OT', color='green', alpha=0.7)
    ax.set_xlabel('Demand Factor', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q3 vs Q2: Balancing Cost', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels([f"{int(f*100)}%" for f in factors])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q3'], 'Q3_vs_Q2.png'), dpi=150)
    plt.close()
    
    return all_tables

# =============================================================================
# Q4 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q4_all(dirs, week=1):
    """Run all Q4 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ4 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Cost Ratio
    ratios = [0.5, 0.75, 1.0, 1.33, 1.5, 2.0]
    results = []
    for ratio in ratios:
        p = BASE_PARAMS.copy()
        p['support_cost'] = p['machine_ot_cost'] * ratio
        r = solve_q4(demand[week], params=p)
        results.append({'Cost_Ratio': ratio, 'c_supp': p['support_cost'], 'Total_Cost': r['objective'],
                       'Total_OT': r['total_overtime'], 'Max_OT': r['max_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q4_CostRatio'] = df
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.plot(df['Cost_Ratio'], df['Total_Cost'], 'bo-', linewidth=2, markersize=8, label='Total Cost')
    ax1.set_xlabel('Cost Ratio (c_supp/c_prod)', fontsize=12)
    ax1.set_ylabel('Total Cost ($)', fontsize=12, color='blue')
    ax2 = ax1.twinx()
    ax2.plot(df['Cost_Ratio'], df['Max_OT'], 'r^--', linewidth=2, markersize=8, label='Max OT')
    ax2.set_ylabel('Max Overtime (hrs)', fontsize=12, color='red')
    ax1.set_title('Q4: Cost and Max OT vs Cost Ratio', fontsize=14)
    fig.legend(loc='upper right', bbox_to_anchor=(0.85, 0.85))
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q4'], 'Q4_CostRatio.png'), dpi=150)
    plt.close()
    
    # 2. c_prod
    costs = [20, 25, 30, 35, 40]
    results = []
    for c in costs:
        p = BASE_PARAMS.copy()
        p['machine_ot_cost'] = c
        r = solve_q4(demand[week], params=p)
        results.append({'c_prod': c, 'Total_Cost': r['objective'], 'OT_Cost': r['overtime_cost'], 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q4_c_prod'] = df
    
    # 3. c_supp
    costs = [30, 40, 50, 60]
    results = []
    for c in costs:
        p = BASE_PARAMS.copy()
        p['support_cost'] = c
        r = solve_q4(demand[week], params=p)
        results.append({'c_supp': c, 'Total_Cost': r['objective'], 'Support_Cost': r['support_cost'], 'Max_OT': r['max_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q4_c_supp'] = df
    
    # 4. Demand
    factors = [0.90, 1.00, 1.10]
    results = []
    for f in factors:
        r = solve_q4(scale_demand(demand[week], f))
        results.append({'Demand_Factor': f, 'Total_Cost': r['objective'], 'OT_Cost': r['overtime_cost'], 'Support_Cost': r['support_cost']})
    df = pd.DataFrame(results)
    all_tables['Q4_Demand'] = df
    
    # 5. Compare Q2, Q3, Q4
    r2 = solve_q2(demand[week])
    r3 = solve_q3(demand[week])
    r4 = solve_q4(demand[week])
    results = [
        {'Model': 'Q2', 'Total_OT': r2['total_overtime'], 'Max_OT': None, 'Cost_30xOT_40xMax': 30*r2['total_overtime']},
        {'Model': 'Q3', 'Total_OT': r3['total_overtime'], 'Max_OT': r3['max_overtime'], 'Cost_30xOT_40xMax': 30*r3['total_overtime']+40*r3['max_overtime']},
        {'Model': 'Q4', 'Total_OT': r4['total_overtime'], 'Max_OT': r4['max_overtime'], 'Cost_30xOT_40xMax': r4['objective']}
    ]
    df = pd.DataFrame(results)
    all_tables['Q4_Comparison'] = df
    
    fig, ax = plt.subplots(figsize=(8, 6))
    models = ['Q2', 'Q3', 'Q4']
    total_ot = [r2['total_overtime'], r3['total_overtime'], r4['total_overtime']]
    ax.bar(models, total_ot, color=['blue', 'green', 'red'], alpha=0.7)
    ax.set_xlabel('Model', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q2 vs Q3 vs Q4: Total Overtime Comparison', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q4'], 'Q4_Comparison.png'), dpi=150)
    plt.close()
    
    return all_tables

# =============================================================================
# Q5 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q5_all(dirs, week=1):
    """Run all Q5 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ5 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Initial Configurations
    results = []
    for name, config in INITIAL_CONFIGS.items():
        r = solve_q5(demand[week], initial_setup=config)
        results.append({'Config': name, 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q5_InitialConfig'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(df['Config'], df['Total_OT'], color='purple', alpha=0.7)
    ax.set_xlabel('Initial Configuration', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q5: Overtime by Initial Setup Configuration', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q5'], 'Q5_InitialConfig.png'), dpi=150)
    plt.close()
    
    # 2. Demand
    factors = [0.90, 1.00, 1.10]
    results = []
    for f in factors:
        r = solve_q5(scale_demand(demand[week], f))
        results.append({'Demand_Factor': f, 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q5_Demand'] = df
    
    # 3. Setup
    reductions = [0.00, 0.25, 0.50]
    results = []
    for red in reductions:
        r = solve_q5(demand[week], setup_df=scale_setup(setup_times_df, red))
        results.append({'Setup_Reduction': red, 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q5_Setup'] = df
    
    # 4. Compare to Q2
    results = []
    r2 = solve_q2(demand[week])
    for name, config in INITIAL_CONFIGS.items():
        r5 = solve_q5(demand[week], initial_setup=config)
        results.append({'Config': name, 'Q2_OT': r2['total_overtime'], 'Q5_OT': r5['total_overtime'],
                       'Savings': r2['total_overtime'] - r5['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q5_vs_Q2'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(INITIAL_CONFIGS))
    width = 0.35
    ax.bar(x - width/2, df['Q2_OT'], width, label='Q2 (No initial setup)', color='blue', alpha=0.7)
    ax.bar(x + width/2, df['Q5_OT'], width, label='Q5 (With initial setup)', color='green', alpha=0.7)
    ax.set_xlabel('Configuration', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q5 vs Q2: Value of Initial Setup', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(list(INITIAL_CONFIGS.keys()))
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q5'], 'Q5_vs_Q2.png'), dpi=150)
    plt.close()
    
    # 5. Yield
    improvements = [0.00, 0.05, 0.10]
    results = []
    for imp in improvements:
        r = solve_q5(demand[week], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q5_Yield'] = df
    
    return all_tables

# =============================================================================
# Q6 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q6_all(dirs, week=11):
    """Run all Q6 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ6 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Penalty
    penalties = [1, 2, 3, 5, 10, 20]
    results = []
    for pen in penalties:
        p = BASE_PARAMS.copy()
        p['penalty_cost'] = pen
        r = solve_q6(demand[week], params=p)
        results.append({'Penalty': pen, 'Total_Cost': r['objective'], 'Total_OT': r['total_overtime'],
                       'Max_OT': r['max_overtime'], 'Shortfall': r['total_shortfall'], 'Shortfall_Cost': r['shortfall_cost']})
    df = pd.DataFrame(results)
    all_tables['Q6_Penalty'] = df
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.plot(df['Penalty'], df['Total_Cost'], 'bo-', linewidth=2, markersize=8, label='Total Cost')
    ax1.set_xlabel('Penalty Cost ($/unit)', fontsize=12)
    ax1.set_ylabel('Total Cost ($)', fontsize=12, color='blue')
    ax2 = ax1.twinx()
    ax2.plot(df['Penalty'], df['Shortfall'], 'r^--', linewidth=2, markersize=8, label='Shortfall')
    ax2.set_ylabel('Shortfall (units)', fontsize=12, color='red')
    ax1.set_title('Q6: Cost and Shortfall vs Penalty', fontsize=14)
    fig.legend(loc='upper right', bbox_to_anchor=(0.85, 0.85))
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q6'], 'Q6_Penalty.png'), dpi=150)
    plt.close()
    
    # 2. Demand
    factors = [0.90, 1.00, 1.10, 1.20]
    results = []
    for f in factors:
        r = solve_q6(scale_demand(demand[week], f))
        results.append({'Demand_Factor': f, 'Total_Cost': r['objective'], 'Total_OT': r['total_overtime'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q6_Demand'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(factors))
    width = 0.35
    ax.bar(x - width/2, df['Total_OT'], width, label='Total OT', color='blue', alpha=0.7)
    ax.bar(x + width/2, df['Shortfall']/100, width, label='Shortfall/100', color='red', alpha=0.7)
    ax.set_xlabel('Demand Factor', fontsize=12)
    ax.set_ylabel('Value', fontsize=12)
    ax.set_title('Q6: Overtime and Shortfall vs Demand', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels([f"{int(f*100)}%" for f in factors])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q6'], 'Q6_Demand.png'), dpi=150)
    plt.close()
    
    # 3. Max Overtime
    ot_values = [36, 48, 60]
    results = []
    for ot in ot_values:
        p = BASE_PARAMS.copy()
        p['max_overtime'] = ot
        p['BIG_M'] = 120 + ot + 2
        r = solve_q6(demand[week], params=p)
        results.append({'Max_OT_Allowed': ot, 'Total_Cost': r['objective'], 'OT_Used': r['total_overtime'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q6_MaxOT'] = df
    
    # 4. Yield
    improvements = [0.00, 0.05, 0.10]
    results = []
    for imp in improvements:
        r = solve_q6(demand[week], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Total_Cost': r['objective'], 'Total_OT': r['total_overtime'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q6_Yield'] = df
    
    # 5. c_prod
    costs = [25, 30, 35]
    results = []
    for c in costs:
        p = BASE_PARAMS.copy()
        p['machine_ot_cost'] = c
        r = solve_q6(demand[week], params=p)
        results.append({'c_prod': c, 'Total_Cost': r['objective'], 'OT_Cost': r['overtime_cost']})
    df = pd.DataFrame(results)
    all_tables['Q6_c_prod'] = df
    
    # 6. c_supp
    costs = [35, 40, 45]
    results = []
    for c in costs:
        p = BASE_PARAMS.copy()
        p['support_cost'] = c
        r = solve_q6(demand[week], params=p)
        results.append({'c_supp': c, 'Total_Cost': r['objective'], 'Support_Cost': r['support_cost']})
    df = pd.DataFrame(results)
    all_tables['Q6_c_supp'] = df
    
    # 7. Demand by Part
    results = []
    for i, part in enumerate(get_parts()):
        for factor in [0.90, 1.00, 1.10]:
            d = demand[week].copy()
            d[i] = int(d[i] * factor)
            r = solve_q6(d)
            results.append({'Part': part, 'Factor': factor, 'Shortfall': r['shortfall_by_part'][part], 'Total_Cost': r['objective']})
    df = pd.DataFrame(results)
    all_tables['Q6_Demand_ByPart'] = df
    
    fig, ax = plt.subplots(figsize=(12, 6))
    pivot = df.pivot(index='Part', columns='Factor', values='Shortfall')
    pivot.plot(kind='bar', ax=ax)
    ax.set_xlabel('Part', fontsize=12)
    ax.set_ylabel('Shortfall (units)', fontsize=12)
    ax.set_title('Q6: Shortfall by Part and Demand Factor', fontsize=14)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q6'], 'Q6_Demand_ByPart.png'), dpi=150)
    plt.close()
    
    return all_tables

# =============================================================================
# Q7 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q7_all(dirs):
    """Run all Q7 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ7 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Week Combinations
    week_pairs = [(1, 2), (11, 12), (3, 4)]
    results = []
    for w1, w2 in week_pairs:
        r = solve_q7(demand[w1], demand[w2])
        results.append({'Weeks': f"{w1}-{w2}", 'Total_OT': r['total_overtime'], 'Week1_OT': r['overtime_week1'],
                       'Week2_OT': r['overtime_week2'], 'Carryovers': r['carryovers']})
    df = pd.DataFrame(results)
    all_tables['Q7_WeekCombos'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(week_pairs))
    width = 0.35
    ax.bar(x - width/2, df['Week1_OT'], width, label='Week 1 OT', color='blue', alpha=0.7)
    ax.bar(x + width/2, df['Week2_OT'], width, label='Week 2 OT', color='green', alpha=0.7)
    ax.set_xlabel('Week Combination', fontsize=12)
    ax.set_ylabel('Overtime (hrs)', fontsize=12)
    ax.set_title('Q7: Overtime by Week Combination', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(df['Weeks'])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q7'], 'Q7_WeekCombos.png'), dpi=150)
    plt.close()
    
    # 2. Demand Pattern
    base = demand[1]
    patterns = [('W1>W2', base, scale_demand(base, 0.85)), ('W1=W2', base, base), ('W1<W2', scale_demand(base, 0.85), base)]
    results = []
    for name, d1, d2 in patterns:
        r = solve_q7(d1, d2)
        results.append({'Pattern': name, 'Total_OT': r['total_overtime'], 'Week1_OT': r['overtime_week1'], 'Week2_OT': r['overtime_week2']})
    df = pd.DataFrame(results)
    all_tables['Q7_DemandPattern'] = df
    
    # 3. Setup
    reductions = [0.00, 0.25, 0.50]
    results = []
    for red in reductions:
        r = solve_q7(demand[1], demand[2], setup_df=scale_setup(setup_times_df, red))
        results.append({'Setup_Reduction': red, 'Total_OT': r['total_overtime'], 'Carryovers': r['carryovers']})
    df = pd.DataFrame(results)
    all_tables['Q7_Setup'] = df
    
    # 4. Initial Config
    results = []
    for name, config in list(INITIAL_CONFIGS.items())[:3]:
        r = solve_q7(demand[1], demand[2], initial_setup=config)
        results.append({'Config': name, 'Total_OT': r['total_overtime'], 'Carryovers': r['carryovers']})
    df = pd.DataFrame(results)
    all_tables['Q7_InitialConfig'] = df
    
    # 5. Compare to 2×Q5
    week_pairs = [(1, 2), (11, 12)]
    results = []
    for w1, w2 in week_pairs:
        r5_w1 = solve_q5(demand[w1])
        r5_w2 = solve_q5(demand[w2])
        q5_total = r5_w1['total_overtime'] + r5_w2['total_overtime']
        r7 = solve_q7(demand[w1], demand[w2])
        results.append({'Weeks': f"{w1}-{w2}", 'Q5x2_OT': q5_total, 'Q7_OT': r7['total_overtime'], 
                       'Carryover_Savings': q5_total - r7['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q7_vs_2xQ5'] = df
    
    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(len(week_pairs))
    width = 0.35
    ax.bar(x - width/2, df['Q5x2_OT'], width, label='2×Q5', color='blue', alpha=0.7)
    ax.bar(x + width/2, df['Q7_OT'], width, label='Q7', color='green', alpha=0.7)
    ax.set_xlabel('Week Combination', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q7 vs 2×Q5: Carryover Value', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(df['Weeks'])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q7'], 'Q7_vs_2xQ5.png'), dpi=150)
    plt.close()
    
    # 6. Yield
    improvements = [0.00, 0.05, 0.10]
    results = []
    for imp in improvements:
        r = solve_q7(demand[1], demand[2], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Total_OT': r['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q7_Yield'] = df
    
    return all_tables

# =============================================================================
# Q8 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q8_all(dirs):
    """Run all Q8 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ8 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Holding Cost
    h_values = [0.5, 1, 2, 3, 5]
    results = []
    for h in h_values:
        p = BASE_PARAMS.copy()
        p['holding_cost'] = h
        r = solve_q8(demand[1], demand[2], params=p)
        results.append({'Holding_Cost': h, 'Objective': r['objective'], 'Total_OT': r['total_overtime'],
                       'Inventory': r['total_inventory'], 'Inv_Cost': r['inventory_cost']})
    df = pd.DataFrame(results)
    all_tables['Q8_HoldingCost'] = df
    
    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.plot(df['Holding_Cost'], df['Objective'], 'bo-', linewidth=2, markersize=8, label='Objective')
    ax1.set_xlabel('Holding Cost ($/unit)', fontsize=12)
    ax1.set_ylabel('Objective Value', fontsize=12, color='blue')
    ax2 = ax1.twinx()
    ax2.plot(df['Holding_Cost'], df['Inventory'], 'r^--', linewidth=2, markersize=8, label='Inventory')
    ax2.set_ylabel('Inventory (units)', fontsize=12, color='red')
    ax1.set_title('Q8: Objective and Inventory vs Holding Cost', fontsize=14)
    fig.legend(loc='upper right', bbox_to_anchor=(0.85, 0.85))
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q8'], 'Q8_HoldingCost.png'), dpi=150)
    plt.close()
    
    # 2. Demand Pattern (W2 spike)
    base = demand[1]
    spikes = [0, 0.10, 0.20]
    results = []
    for spike in spikes:
        d2 = scale_demand(base, 1 + spike)
        r = solve_q8(base, d2)
        results.append({'W2_Spike': spike, 'Total_OT': r['total_overtime'], 'Inventory': r['total_inventory']})
    df = pd.DataFrame(results)
    all_tables['Q8_DemandPattern'] = df
    
    # 3. Setup
    reductions = [0.00, 0.25, 0.50]
    results = []
    for red in reductions:
        r = solve_q8(demand[1], demand[2], setup_df=scale_setup(setup_times_df, red))
        results.append({'Setup_Reduction': red, 'Total_OT': r['total_overtime'], 'Inventory': r['total_inventory']})
    df = pd.DataFrame(results)
    all_tables['Q8_Setup'] = df
    
    # 4. Max Overtime
    ot_values = [36, 48, 60]
    results = []
    for ot in ot_values:
        p = BASE_PARAMS.copy()
        p['max_overtime'] = ot
        p['BIG_M'] = 120 + ot + 2
        r = solve_q8(demand[1], demand[2], params=p)
        results.append({'Max_OT': ot, 'Total_OT_Used': r['total_overtime'], 'Inventory': r['total_inventory']})
    df = pd.DataFrame(results)
    all_tables['Q8_MaxOT'] = df
    
    # 5. Compare to Q7
    h_values = [1, 2, 3]
    results = []
    r7 = solve_q7(demand[1], demand[2])
    for h in h_values:
        p = BASE_PARAMS.copy()
        p['holding_cost'] = h
        r8 = solve_q8(demand[1], demand[2], params=p)
        results.append({'Holding_Cost': h, 'Q7_OT': r7['total_overtime'], 'Q8_OT': r8['total_overtime'],
                       'Q8_Inventory': r8['total_inventory'], 'OT_Savings': r7['total_overtime'] - r8['total_overtime']})
    df = pd.DataFrame(results)
    all_tables['Q8_vs_Q7'] = df
    
    fig, ax = plt.subplots(figsize=(8, 6))
    x = np.arange(len(h_values))
    width = 0.35
    ax.bar(x - width/2, df['Q7_OT'], width, label='Q7 (No Inventory)', color='blue', alpha=0.7)
    ax.bar(x + width/2, df['Q8_OT'], width, label='Q8 (With Inventory)', color='green', alpha=0.7)
    ax.set_xlabel('Holding Cost ($/unit)', fontsize=12)
    ax.set_ylabel('Total Overtime (hrs)', fontsize=12)
    ax.set_title('Q8 vs Q7: Value of Inventory', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels([f"${h}" for h in h_values])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q8'], 'Q8_vs_Q7.png'), dpi=150)
    plt.close()
    
    # 6. Yield
    improvements = [0.00, 0.05, 0.10]
    results = []
    for imp in improvements:
        r = solve_q8(demand[1], demand[2], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Total_OT': r['total_overtime'], 'Inventory': r['total_inventory']})
    df = pd.DataFrame(results)
    all_tables['Q8_Yield'] = df
    
    # 7. Inventory by Part
    r = solve_q8(demand[1], demand[2])
    results = [{'Part': p, 'Inventory': r['inventory_by_part'][p]} for p in get_parts()]
    df = pd.DataFrame(results)
    all_tables['Q8_Inventory_ByPart'] = df
    
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(df['Part'], df['Inventory'], color='teal', alpha=0.7)
    ax.set_xlabel('Part', fontsize=12)
    ax.set_ylabel('Inventory (units)', fontsize=12)
    ax.set_title('Q8: Inventory by Part', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q8'], 'Q8_Inventory_ByPart.png'), dpi=150)
    plt.close()
    
    return all_tables

# =============================================================================
# Q9 SENSITIVITY ANALYSIS
# =============================================================================

def sa_q9_all(dirs):
    """Run all Q9 sensitivity analyses with export"""
    print("\n" + "="*70 + "\nQ9 SENSITIVITY ANALYSIS\n" + "="*70)
    all_tables = {}
    
    # 1. Penalty
    penalties = [1, 3, 5, 10]
    results = []
    for pen in penalties:
        p = BASE_PARAMS.copy()
        p['penalty_cost'] = pen
        r = solve_q9(demand[11], demand[12], params=p)
        results.append({'Penalty': pen, 'Total_Cost': r['objective'], 'Shortfall': r['total_shortfall'], 'Inventory': r['total_inventory']})
    df = pd.DataFrame(results)
    all_tables['Q9_Penalty'] = df
    
    # 2. Holding Cost
    h_values = [1, 2, 3, 5]
    results = []
    for h in h_values:
        p = BASE_PARAMS.copy()
        p['holding_cost'] = h
        r = solve_q9(demand[11], demand[12], params=p)
        results.append({'Holding_Cost': h, 'Total_Cost': r['objective'], 'Inventory': r['total_inventory'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q9_HoldingCost'] = df
    
    # 3. c_prod
    costs = [25, 30, 35]
    results = []
    for c in costs:
        p = BASE_PARAMS.copy()
        p['machine_ot_cost'] = c
        r = solve_q9(demand[11], demand[12], params=p)
        results.append({'c_prod': c, 'Total_Cost': r['objective'], 'OT_Cost': r['overtime_cost']})
    df = pd.DataFrame(results)
    all_tables['Q9_c_prod'] = df
    
    # 4. c_supp
    costs = [35, 40, 50]
    results = []
    for c in costs:
        p = BASE_PARAMS.copy()
        p['support_cost'] = c
        r = solve_q9(demand[11], demand[12], params=p)
        results.append({'c_supp': c, 'Total_Cost': r['objective'], 'Support_Cost': r['support_cost']})
    df = pd.DataFrame(results)
    all_tables['Q9_c_supp'] = df
    
    # 5. Demand
    factors = [0.90, 1.00, 1.10]
    results = []
    for f in factors:
        r = solve_q9(scale_demand(demand[11], f), scale_demand(demand[12], f))
        results.append({'Demand_Factor': f, 'Total_Cost': r['objective'], 'Total_OT': r['total_overtime'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q9_Demand'] = df
    
    # 6. Max Overtime
    ot_values = [36, 48, 60]
    results = []
    for ot in ot_values:
        p = BASE_PARAMS.copy()
        p['max_overtime'] = ot
        p['BIG_M'] = 120 + ot + 2
        r = solve_q9(demand[11], demand[12], params=p)
        results.append({'Max_OT': ot, 'Total_Cost': r['objective'], 'OT_Used': r['total_overtime'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q9_MaxOT'] = df
    
    # 7. Yield
    improvements = [0.00, 0.05, 0.10]
    results = []
    for imp in improvements:
        r = solve_q9(demand[11], demand[12], yields=scale_yields(yields_base, imp))
        results.append({'Yield_Improvement': imp, 'Total_Cost': r['objective'], 'Shortfall': r['total_shortfall']})
    df = pd.DataFrame(results)
    all_tables['Q9_Yield'] = df
    
    # 8. 2D: Penalty × Holding Cost
    penalties = [1, 3, 5, 10]
    holdings = [1, 2, 3, 5]
    results = []
    for pen in penalties:
        for h in holdings:
            p = BASE_PARAMS.copy()
            p['penalty_cost'] = pen
            p['holding_cost'] = h
            r = solve_q9(demand[11], demand[12], params=p)
            results.append({'Penalty': pen, 'Holding': h, 'Total_Cost': r['objective'], 
                           'Shortfall': r['total_shortfall'], 'Inventory': r['total_inventory']})
    df = pd.DataFrame(results)
    all_tables['Q9_2D_Penalty_Holding'] = df
    
    # Heatmap for 2D sensitivity
    pivot_cost = df.pivot(index='Penalty', columns='Holding', values='Total_Cost')
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(pivot_cost, annot=True, fmt='.0f', cmap='YlOrRd', ax=ax)
    ax.set_xlabel('Holding Cost ($/unit)', fontsize=12)
    ax.set_ylabel('Penalty Cost ($/unit)', fontsize=12)
    ax.set_title('Q9: Total Cost - Penalty × Holding Cost', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q9'], 'Q9_2D_Penalty_Holding_Cost.png'), dpi=150)
    plt.close()
    
    pivot_short = df.pivot(index='Penalty', columns='Holding', values='Shortfall')
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(pivot_short, annot=True, fmt='.0f', cmap='Blues', ax=ax)
    ax.set_xlabel('Holding Cost ($/unit)', fontsize=12)
    ax.set_ylabel('Penalty Cost ($/unit)', fontsize=12)
    ax.set_title('Q9: Shortfall - Penalty × Holding Cost', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q9'], 'Q9_2D_Penalty_Holding_Shortfall.png'), dpi=150)
    plt.close()
    
    # 9. Compare to Q6, Q7, Q8
    r6_w1 = solve_q6(demand[11])
    r6_w2 = solve_q6(demand[12])
    r7 = solve_q7(demand[11], demand[12])
    r8 = solve_q8(demand[11], demand[12])
    r9 = solve_q9(demand[11], demand[12])
    results = [
        {'Model': 'Q6 (W11+W12)', 'Total_OT': r6_w1['total_overtime']+r6_w2['total_overtime'], 
         'Shortfall': r6_w1['total_shortfall']+r6_w2['total_shortfall'], 'Inventory': 0},
        {'Model': 'Q7', 'Total_OT': r7['total_overtime'], 'Shortfall': 0, 'Inventory': 0},
        {'Model': 'Q8', 'Total_OT': r8['total_overtime'], 'Shortfall': 0, 'Inventory': r8['total_inventory']},
        {'Model': 'Q9', 'Total_OT': r9['total_overtime'], 'Shortfall': r9['total_shortfall'], 'Inventory': r9['total_inventory']}
    ]
    df = pd.DataFrame(results)
    all_tables['Q9_Comparison'] = df
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(results))
    width = 0.25
    ax.bar(x - width, df['Total_OT'], width, label='Total OT', color='blue', alpha=0.7)
    ax.bar(x, df['Shortfall']/100, width, label='Shortfall/100', color='red', alpha=0.7)
    ax.bar(x + width, df['Inventory']/100, width, label='Inventory/100', color='green', alpha=0.7)
    ax.set_xlabel('Model', fontsize=12)
    ax.set_ylabel('Value (scaled)', fontsize=12)
    ax.set_title('Q9 vs Q6, Q7, Q8: Model Comparison', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(df['Model'])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(dirs['q9'], 'Q9_Comparison.png'), dpi=150)
    plt.close()
    
    return all_tables

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def run_all_and_export():
    """Run all sensitivity analyses and export results"""
    print("\n" + "="*80)
    print(" "*15 + "FDC COMPLETE SENSITIVITY ANALYSIS WITH EXPORT")
    print("="*80)
    
    # Setup directories
    dirs = setup_output_dirs()
    
    # Run all analyses
    all_tables = {}
    all_tables.update(sa_q2_all(dirs, week=1))
    all_tables.update(sa_q3_all(dirs, week=1))
    all_tables.update(sa_q4_all(dirs, week=1))
    all_tables.update(sa_q5_all(dirs, week=1))
    all_tables.update(sa_q6_all(dirs, week=11))
    all_tables.update(sa_q7_all(dirs))
    all_tables.update(sa_q8_all(dirs))
    all_tables.update(sa_q9_all(dirs))
    
    # Export all tables to CSV
    print("\n" + "="*70 + "\nEXPORTING TABLES\n" + "="*70)
    for name, df in all_tables.items():
        filepath = os.path.join(dirs['tables'], f"{name}.csv")
        df.to_csv(filepath, index=False)
        print(f"  Saved: {name}.csv")
    
    # Export all tables to single Excel file
    excel_path = os.path.join(dirs['tables'], "All_Sensitivity_Tables.xlsx")
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        for name, df in all_tables.items():
            sheet_name = name[:31]  # Excel sheet names limited to 31 chars
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"\n  Combined Excel file: All_Sensitivity_Tables.xlsx")
    
    # Summary
    print("\n" + "="*80)
    print(" "*25 + "EXPORT COMPLETE")
    print("="*80)
    print(f"\nOutput Directory: {dirs['base']}")
    print(f"  - Tables: {len(all_tables)} CSV files + 1 Excel file")
    print(f"  - Graphs: Multiple PNG files in Q2-Q9 subfolders")
    print("\nGraph counts:")
    for q in ['q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9']:
        count = len([f for f in os.listdir(dirs[q]) if f.endswith('.png')])
        print(f"  {q.upper()}: {count} graphs")
    
    return dirs, all_tables

if __name__ == "__main__":
    dirs, tables = run_all_and_export()


               FDC COMPLETE SENSITIVITY ANALYSIS WITH EXPORT
Output directory created: FDC_Sensitivity_Analysis_20251210_084532

Q2 SENSITIVITY ANALYSIS

Q3 SENSITIVITY ANALYSIS

Q4 SENSITIVITY ANALYSIS

Q5 SENSITIVITY ANALYSIS

Q6 SENSITIVITY ANALYSIS

Q7 SENSITIVITY ANALYSIS

Q8 SENSITIVITY ANALYSIS

Q9 SENSITIVITY ANALYSIS

EXPORTING TABLES
  Saved: Q2_Demand_Overall.csv
  Saved: Q2_Demand_ByPart.csv
  Saved: Q2_Yield.csv
  Saved: Q2_Setup.csv
  Saved: Q2_MaxOT.csv
  Saved: Q3_Demand.csv
  Saved: Q3_Setup.csv
  Saved: Q3_Yield.csv
  Saved: Q3_vs_Q2.csv
  Saved: Q4_CostRatio.csv
  Saved: Q4_c_prod.csv
  Saved: Q4_c_supp.csv
  Saved: Q4_Demand.csv
  Saved: Q4_Comparison.csv
  Saved: Q5_InitialConfig.csv
  Saved: Q5_Demand.csv
  Saved: Q5_Setup.csv
  Saved: Q5_vs_Q2.csv
  Saved: Q5_Yield.csv
  Saved: Q6_Penalty.csv
  Saved: Q6_Demand.csv
  Saved: Q6_MaxOT.csv
  Saved: Q6_Yield.csv
  Saved: Q6_c_prod.csv
  Saved: Q6_c_supp.csv
  Saved: Q6_Demand_ByPart.csv
  Saved: Q7_WeekCombos.csv
  